In [1]:
import numpy as np
from packages.Utility.CNN.Callbacks import *
from packages.Utility.DatasetManager import DatasetReader
from packages.Utility.Gabor import GaborFilterBanks, GaborParams
from packages.Model.LeNet import LeNet

Using TensorFlow backend.


In [21]:
fold_index = 2
dataset_dir = 'dataset/bin/DG.32.A.50K'
trained_dir = 'trained/DG.32.A.50K LN1.1'

# Memuat Dataset

In [22]:
reader = DatasetReader(dataset_dir)

X_train, Y_train_onehot, Y_train_single = reader.getTrainData(fold_index)
X_test, Y_test_onehot, Y_test_single = reader.getTestData(fold_index)

# Inisiasi Callbacks

In [23]:
val_model = ValidateModel(X_train, Y_train_onehot)
save_model = SaveModel(val_model, base_dir = trained_dir, fold = fold_index + 1, min_acc = 0.1)
stop_train = StopTraining(val_model)
history_save = HistorySaver(val_model, base_dir = trained_dir, fold = fold_index + 1)

# Parameter Gabor

In [24]:
class RotatedGaborParams(GaborParams):

    def __init__(self, lambd, sigma, psi, gamma):
        self.lambd = lambd
        self.sigma = sigma
        self.psi = psi,
        self.gamma = gamma

    def getParams(self, shape, i, j):
        multiplier = 360 / shape[3]
        degree = i * multiplier
        theta = degree * np.pi / 180
        
        return self.lambd, theta, self.psi, self.sigma, self.gamma

# Mendefinisikan Kernel Gabor

In [25]:
kernel1 = GaborFilterBanks(RotatedGaborParams(psi = 1, gamma = 0.5, lambd = 5, sigma = 3))
kernel2 = GaborFilterBanks(RotatedGaborParams(psi = 1, gamma = 0.75, lambd = 3, sigma = 5))

# Mendefinisikan Model CNN
Mendefinisikan arsitektur CNN

In [26]:
ars = LeNet(kernels = {
    1: kernel1,
    2: kernel2
}, loss = 'categorical_crossentropy')
ars.save(f'{trained_dir}/Fold {fold_index + 1}/init.h5')

model = ars.model
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
c1 (Conv2D)                  (None, 28, 28, 6)         156       
_________________________________________________________________
mp1 (MaxPooling2D)           (None, 14, 14, 6)         0         
_________________________________________________________________
c2 (Conv2D)                  (None, 10, 10, 16)        2416      
_________________________________________________________________
mp2 (MaxPooling2D)           (None, 5, 5, 16)          0         
_________________________________________________________________
flatten (Flatten)            (None, 400)               0         
_________________________________________________________________
d1 (Dense)                   (None, 120)               48120     
_________________________________________________________________
d2 (Dense)                   (None, 84)                10164     
__________

# Melakukan Pelatihan
Melakukan pelatihan pada CNN

In [27]:
ars.compile()
history = model.fit(X_train, Y_train_onehot, epochs = 300, verbose = 3, batch_size = 128, callbacks=[val_model, save_model, history_save, stop_train], initial_epoch=0)

Epoch 1/300
New best accuracy : 0.8267200000190735
Model saved to disk
Passes : 0/50
Epoch 2/300
New best accuracy : 0.8820800000063578
Model saved to disk
Passes : 0/50
Epoch 3/300
New best accuracy : 0.8995466666475932
Model saved to disk
Passes : 0/50
Epoch 4/300
New best accuracy : 0.9078399999809265
Model saved to disk
Passes : 0/50
Epoch 5/300
New best accuracy : 0.9144000000127157
Model saved to disk
Passes : 0/50
Epoch 6/300
New best accuracy : 0.9183199999936422
Model saved to disk
Passes : 0/50
Epoch 7/300
New best accuracy : 0.9210666666793823
Model saved to disk
Passes : 0/50
Epoch 8/300
New best accuracy : 0.9232533333269755
Model saved to disk
Passes : 0/50
Epoch 9/300
New best accuracy : 0.9269866666730245
Model saved to disk
Passes : 0/50
Epoch 10/300
New best accuracy : 0.9292800000127157
Model saved to disk
Passes : 0/50
Epoch 11/300
New best accuracy : 0.9303733333142599
Model saved to disk
Passes : 0/50
Epoch 12/300
New best accuracy : 0.9316533333142598
Model saved